In [5]:
from types import SimpleNamespace
from recsys.io.service import IOService

In [6]:
filepath = "tests/data/test_workflow_config.yml"
config = IOService.read(filepath)
config = SimpleNamespace(**config)
config.pipeline


{'name': 'movielens25m_etl',
 'description': 'Extracts, transforms and loads the GroupLens Movielens25M ratings dataset',
 'tasks': [{'task': {'name': 'source_movielens25m',
    'description': 'Obtain Movielens25M Dataset from the Grouplens website',
    'operator': {'name': 'TestOperator1',
     'module': 'tests.data.operators',
     'params': {'source': 'https://files.grouplens.org/datasets/movielens/ml-25m.zip',
      'download': 'data/movielens25m/ext/',
      'extract': 'data/movielens25m/raw/'}}}},
  {'task': {'name': 'spotify',
    'description': 'Spotify Million Playlist Benchmark Dataset',
    'operator': {'name': 'TestOperator2',
     'module': 'tests.data.operators',
     'params': {'uri1': 'https://www.aicrowd.com/challenges/spotify-million-playlist-dataset-challenge/dataset_files?unique_download_uri=204765&challenge_id=277',
      'uri2': 'https://www.aicrowd.com/challenges/spotify-million-playlist-dataset-challenge/dataset_files?unique_download_uri=204766&challenge_id=277